In [1]:
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import v2

F:\anaconda\anaconda3\envs\DeltaZero\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
F:\anaconda\anaconda3\envs\DeltaZero\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you m

In [2]:
train_data = pd.read_csv("data/train.csv")
train_data

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera
...,...,...
18348,images/18348.jpg,aesculus_glabra
18349,images/18349.jpg,liquidambar_styraciflua
18350,images/18350.jpg,cedrus_libani
18351,images/18351.jpg,prunus_pensylvanica


In [3]:
class LeaveDataset(Dataset):
    def __init__(self, x, y, train=True):
        self.x = x
        self.y = y
        self.train = train
        self.train_transpose = v2.Compose([
            v2.RandomHorizontalFlip(),
            v2.RandomVerticalFlip(),
            v2.RandomRotation(180, fill=(255, 255, 255)),
            v2.ColorJitter(0.5),
            v2.Resize((224, 224)),
            v2.PILToTensor(),
            v2.ToDtype(torch.float32, True),
            v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.test_transpose = v2.Compose([
            v2.Resize((224, 224)),
            v2.PILToTensor(),
            v2.ToDtype(torch.float32, True),
            v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        self.data_path = "data/images/"
        
    def __getitem__(self, idx):
        x = self.x[idx]
        y = self.y[idx]
        x = Image.open(self.data_path + x)
        x = self.train_transpose(x) if self.train else self.test_transpose(x)
        return x, y
    
    def __len__(self):
        return len(self.x)

In [4]:
# 加载训练数据集
x, y = train_data['image'], train_data['label']
x, y

(0            images/0.jpg
 1            images/1.jpg
 2            images/2.jpg
 3            images/3.jpg
 4            images/4.jpg
                ...       
 18348    images/18348.jpg
 18349    images/18349.jpg
 18350    images/18350.jpg
 18351    images/18351.jpg
 18352    images/18352.jpg
 Name: image, Length: 18353, dtype: object,
 0               maclura_pomifera
 1               maclura_pomifera
 2               maclura_pomifera
 3               maclura_pomifera
 4               maclura_pomifera
                   ...           
 18348            aesculus_glabra
 18349    liquidambar_styraciflua
 18350              cedrus_libani
 18351        prunus_pensylvanica
 18352            quercus_montana
 Name: label, Length: 18353, dtype: object)

In [5]:
train_x, valid_x, train_y, valid_y = train_

NameError: name 'LabelEncoder' is not defined